# CPA Analysis Notebook

This notebook uses various captured data and gives CPA results.

In [ ]:
import scared
import estraces
import numpy as np

In [ ]:
knownkey = [43, 126, 21, 22, 40, 174, 210, 166, 171, 247, 21, 136, 9, 207, 79, 60]

In [ ]:
def find_success(att, knownkey):
    recovered_masterkey = np.argmax(att.scores, axis=0).astype('uint8')
    print(recovered_masterkey == knownkey)
    
    for i in range(0, np.shape(att.convergence_traces)[2]):
        if np.all(np.argmax(att.convergence_traces[:,:,i], axis=0) == knownkey):
            return (i+1)*att.convergence_step
    return -1

In [ ]:
import scared
from scipy import signal


def do_cpa(filename, step=25, window=None, hpf=None, lpf=None):
    ths = estraces.read_ths_from_ets_file(filename)
    print(ths)

    selection_function = scared.aes.selection_functions.encrypt.FirstSubBytes()

    att = scared.CPAAttack(selection_function=selection_function,
                           model=scared.HammingWeight(), 
                           discriminant=scared.maxabs,
                           convergence_step=step)
    
    processes = []
    
    # Simple high pass filter (HPF)
    if hpf:
        sos = signal.butter(5, hpf, 'highpass', output='sos')
        
        @scared.preprocess
        def hpfproc(traces):
            return signal.sosfilt(sos, traces)

        processes.append(hpfproc)
    
    # Simple low pass filter (LPF)
    if lpf:
        sos = signal.butter(5, lpf, 'lowpass', output='sos')
        
        @scared.preprocess
        def lpfproc(traces):
            return signal.sosfilt(sos, traces)

        processes.append(lpfproc)
        
    # This resync preprocessing isn't really used, was used with some JTAG captures only
    @scared.preprocess
    def resync(traces):
        import numpy as np
        first = []
        for w in traces:
            first.append(w[0])
        avg = np.mean(first)
        
        tracesresync = []
        for w in traces:
            if w[0] < avg:
                tracesresync.append(w[:-1])
            else:
                tracesresync.append(w[1:])                
        return np.array(tracesresync)
    #processes.append(resync)
    
    if window:
        container = scared.Container(ths, frame=range(window[0], window[1]), preprocesses=processes)
    else:
        container = scared.Container(ths, preprocesses=processes)
        

    att.run(container)

    import numpy as np

    recovered_masterkey = np.argmax(att.scores, axis=0).astype('uint8')

    return att

In [ ]:
#Uncomment relevant datasets
filenames = [('d:/data_store/baseline120mhz_sync_120msps_cpa.ets', 25, None, None, None),
             ('d:/data_store/baseline120mhz_sync_60msps_cpa.ets', 25, None, None, None),
             ('d:/data_store/baseline120mhz_async_120msps_cpa.ets', 125, None, None, None),
             ('d:/data_store/baseline120mhz_async_60msps_cpa.ets', 125, (0, 3000), None, None),
             ('d:/data_store/baseline15mhz_sync_120msps_cpa.ets', 50, None, None, None),
             ('d:/data_store/baseline15mhz_sync_60msps_cpa.ets', 50, None, None, None),
             ('d:/data_store/baseline15mhz_async_120msps_cpa.ets', 50, (1000, 6000), None, None),
             ('d:/data_store/baseline15mhz_async_60msps_cpa.ets', 50, (500, 3000), None, None),
             #('d:/data_store/baseline2mhz_async_120msps_cpa.ets', 50, None, None, None),
             #('d:/data_store/baseline2mhz_async_60msps_cpa.ets', 50, None, None), #(60000, 80000), None),
             #('d:/data_store/baseline2mhz_async_60msps_cpa.ets', None, None, None),
             #('d:/data_store/mixer_default_120mhz_sync_120msps_cpa.ets', 25, None, None, None),
             #('d:/data_store/mixer_slowerfilter_120mhz_sync_120msps_cpa.ets', 25, None, None, None),
             #('d:/data_store/vdiv_120mhz_sync_120msps_cpa.ets', 25, None, 0.1, None),    
             ('d:/data_store/opticalisolator_mixer_120mhz_sync_120msps_cpa.ets', 25, None, None, None),
             ('d:/data_store/cwhusky_jtag_mixer_15mhzcpu_30mhzadc_sync_k.ets', 25, None, None, None), # 8900 @ 25 steps
             ('d:/data_store/cwhusky_jtagclockoff_mixer_15mhzcpu_30mhzadc_sync.ets', 500, None, None, None),
             ('d:/data_store/cwhusky_shunt_15mhzcpu_30mhzadc_sync.ets', 100, None, None, None),
             #('d:/data_store/cwhusky_jtagcounetmeasured_mixer_15mhzcpu_30mhzadc_sync_k.ets', 25, None, None, None), # 1800@25 steps
             #('d:/data_store/jtagglitch_15MHzCPU_158MHzLA_mixerjtagboard_jtaglaresults_k.ets', 500, (8000, 14000), None, None),
             #('d:/data_store/jtagglitch_15MHzCPU_174MHz_shuntcpa.ets', 500, (17590, 24000), None, None),
             #('d:/data_store/jtagglitch_15MHzCPU_158MHz_shuntcpa.ets', 100, None, None, None),
             #('d:/data_store/jtagglitch_15MHzCPU_159MHz_10koffsetLA_mixerjtagboard_jtaglaresults_60k.ets', 500, (6000,12500), None, None),
             ('d:/data_store/jtagglitch_15MHzCPU_159MHz_async_10koffsetLA_mixerjtagboard_jtaglaresults_k.ets', 500, (3600,9000), None, None),
            #(r'd:/data_store/jtag_taps/mpc5676r_sca_16mhz_32msps_5k_cpa_25k.ets', 500, (5000, 15000), None, None)
            ]

## Datasets that weren't used but still valid
#('d:/data_store/cwhusky_shunt_15mhzcpu_15mhzadc_sync.ets', 25, None, None, None), # 475 @25 steps
#('d:/data_store/cwhusky_jtag_mixer_15mhzcpu_15mhzadc_sync_40k.ets', 25, (1500, 2500), 0.001, None), #32750 @25 steps
#('d:/data_store/cwhusky_jtagclockoff_mixer_15mhzcpu_15mhzadc_sync_100k.ets', 500, (1500, 2500), 0.001, None), #No result

## Datasets that weren't really successful, used for R&D, TODO for future papers
#('d:/data_store/mpc5676r_shunt_64msps_save.ets', 500, None, 0.015, None),
#('d:/data_store/mpc5676r_jtag_etpu_16mhz_64msps_50k_cpa.ets', 500, None, None, None),
#('d:/data_store/mpc5676r_jtag_etpu_16mhz_32msps_150k_cpa.ets', 500, (19000, 20000), 0.015, None),
#('d:/data_store/mpc5676r_sca_16mhz_32msps_5k_cpa_25k.ets', 500, (5000, 15000), None, None),
#('d:/data_store/mpc5676r_sca_jtagclockon_16mhz_32msps_21k_cpa_25k.ets', 500, (5000, 15000), 0.015, 0.3),
#('d:/data_store/mpc5676r_jtag_16mhz_32msps_5koffset_cpa_300k.ets', 500, None, 0.1, 0.5),

In [ ]:
for f in filenames:
    att = do_cpa(f[0], f[1], f[2], f[3], f[4])
    successnum = find_success(att, knownkey)
    
    print("{:s} = {:d}".format(f[0], successnum))
    
    np.save(f[0] + ".npy", att)

The following will plot some information, this was used normally by commenting out the list so online a single thing would run:

In [ ]:
import matplotlib.pylab as plt
from cycler import cycler
for bindex in range(0, 16):
    #bindex = 1
    plt.figure()
    ax = plt.gca()
    ax.set_prop_cycle(cycler('color', [(1,0,0,1.0) if i == knownkey[bindex] else (.5,.5,.5, 0.05) for i in range(0,256)]))
    plt.title('Convergence score', fontsize=20)
    plt.xlabel('Number of blocks', fontsize=14)
    plt.ylabel('CPA score', fontsize=14)
    plt.plot(att.convergence_traces[:, bindex,:].T,)
    plt.show()

In [ ]:
%matplotlib inline
import matplotlib.pylab as plt
from cycler import cycler

plt.figure(figsize=(8, 5))
plt.title('Convergence Scope')#, fontsize=20)
plt.xlabel('Number of Traces')#, fontsize=14)
plt.ylabel('CPA Result')#, fontsize=14)

tcol = [(1,0,0,1.0)]*16

conv_shape = np.shape(att.convergence_traces[0])

for bindex in range(0, 16):
    ax = plt.gca()
    tc = tcol[bindex]
    ax.set_prop_cycle(cycler('color', [(1,0,0,0) if i == knownkey[bindex] else (.5,.5,.5, 0.2) for i in range(0,256)]))
    plt.plot(att.convergence_traces[:, bindex,:].T,)

for bindex in range(0, 16):
    ax = plt.gca()
    tc = tcol[bindex]
    #ax.set_prop_cycle(cycler('color', [tc if i == knownkey[bindex] else (.5,.5,.5, 0.05) for i in range(0,256)]))
    plt.plot(att.convergence_traces[knownkey[bindex], bindex, :].T, 'r')
    
tickstep = int(conv_shape[1] / 10)
    
plt.plot([5000/att.convergence_step, 5000/att.convergence_step], [0.01, 0.08], 'k--')
    
xticks = [j for j in range(0, conv_shape[1], tickstep)]
#xticks.append(conv_shape[1])
xlabels = [str(i * att.convergence_step) for i in xticks]
plt.xticks(xticks, xlabels, rotation=-45)
ax.set_ylim([0.01, 0.08])
    
plt.show()

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt
plt.figure()
for i in range(0, 15):
    plt.plot(att.results[knownkey[i]][i])

## R&D Stuff / Individual Tests

In [ ]:
recovered_masterkey = np.argmax(att.scores, axis=0).astype('uint8')

In [ ]:
recovered_masterkey

In [ ]:
import chipwhisperer as cw
data_dir = "d:/data_store"

In [ ]:
%run "functions.ipynb"

In [ ]:
from scipy.stats import ttest_ind
import matplotlib.pylab as plt

file = "shunt_clkin_clkout_120mhz_sync_120msps_ttest_10k"

start = 100
end = 10000

group1, group2, N = nptload(file)
t_val = ttest_ind(group1[start:end], group2[start:end], axis=0, equal_var=False)[0]
max_t = max(abs(t_val))   

In [ ]:
print(max_t)

In [ ]:
import matplotlib.pylab as plt

plt.figure()
plt.plot(np.mean(group1, axis=0))
plt.show()

In [ ]:
import estraces
ths = estraces.read_ths_from_ets_file(r"d:\data_store\cwhusky_jtagclockoff_mixer_15mhzcpu_30mhzadc_sync.ets")

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt
import numpy as np
plt.plot(np.mean(ths.samples[0:50], axis=0))


In [ ]:
plt.plot(ths.samples[0])
plt.plot(ths.samples[1])
